# 1. 문서의 내용을 읽는다
# 2. 문서를 쪼갠다
#     - 토큰수 초과로 답변을 생성못할 수 있음
#     - 문서가 길면 답변 생성이 오래걸림
# 3. 임베딩 -> 벡터 데이터베이스에 저장
# 4. 질문이 있을때, 벡터 디비에 유사도 검사
# 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
%pip install python-docx

In [ ]:
# 1. 문서의 내용을 읽는다
from docx import Document

document = Document('./tax.docx')
# print(f'document == {document}')
print(f'document == {dir(document)}')
full_text = ''
for index, paragraph in enumerate(document.paragraphs):
    print(f'paragraph == {paragraph.text}') # 문단의 내용을 출력한다
    full_text += paragraph.text





## 2. 문서를 쪼갠다

In [ ]:
%pip install tiktoken


In [10]:
import tiktoken

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model('gpt-4o')
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk_encoding = total_encoding[i:i+chunk_size]
        chunk_text = encoder.decode(chunk_encoding)
        text_list.append(chunk_text)
    return text_list



In [ ]:
chunk_list = split_text(full_text, 1500)


## 3. 문서 임베딩


In [ ]:
%pip install chromadb

In [16]:
import chromadb

chroma_client = chromadb.Client()



In [17]:
# 콜렉션은 RDB에서 테이블
collection_name = 'tax_collection'
tax_collection = chroma_client.create_collection(collection_name)


In [19]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key=openai_key, model_name='text-embedding-3-large')

In [20]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

In [21]:
id_list = []
for idx in range(len(chunk_list)):
    id_list.append(f'{idx}')
    

In [22]:
len(id_list)

102

In [23]:
len(chunk_list)

102

In [24]:
tax_collection.add(documents=chunk_list, ids=id_list)

## 4. 유사도 검색

In [28]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_doc = tax_collection.query(query_texts=query, n_results=3)

In [29]:
retrieved_doc

{'ids': [['5', '76', '13']],
 'embeddings': None,
 'documents': [['법」, 「사립학교교직원 연금법」, 「별정우체국법」 또는 「국민연금과 직역연금의 연계에 관한 법률」(이하 “공적연금 관련법”이라 한다)에 따라 받는 유족연금ㆍ퇴직유족연금ㆍ퇴역유족연금ㆍ장해유족연금ㆍ상이유족연금ㆍ순직유족연금ㆍ직무상유족연금ㆍ위험직무순직유족연금, 장애연금, 장해연금ㆍ비공무상 장해연금ㆍ비직무상 장해연금, 상이연금(傷痍年金), 연계노령유족연금 또는 연계퇴직유족연금나. 삭제<2013. 1. 1.>다. 「산업재해보상보험법」에 따라 받는 각종 연금라. 「국군포로의 송환 및 대우 등에 관한 법률」에 따른 국군포로가 받는 연금마. 삭제<2013. 1. 1.>5. 기타소득 중 다음 각 목의 어느 하나에 해당하는 소득가. 「국가유공자 등 예우 및 지원에 관한 법률」 또는 「보훈보상대상자 지원에 관한 법률」에 따라 받는 보훈급여금ㆍ학습보조비 및 「북한이탈주민의 보호 및 정착지원에 관한 법률」에 따라 받는 정착금ㆍ보로금(報勞金)과 그 밖의 금품나. 「국가보안법」에 따라 받는 상금과 보로금다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용자등 또는 산학협력단으로부터 지급받거나 대학의 학생이 소속 대학에 설치된 산학협력단으로부터 받는 직무발명보상금으로서 대통령령으로 정하는 금액. 다만, 직무발명보상금을 지급한 사용자등 또는 산학협력단과 대통령령으로 정하는 특수관계에 있는 자가 받는 직무발명보상금은 제외한다.마. 「국군포로의 송환 및 대우 등에 관한 법률」에 따라 국군포로가 받는 위로지원금과 그 밖의 금품바. 「문화유산의 보존 및 활용에 관한 법률」에 따라 국가지정문화유산으로 지정된 서화ㆍ골동품의 양도로 발생하는 소득사. 서화ㆍ골동품을 박물관 또는 미술관에 양도함으로써 발생하는 소득아. 제21조제1항제26호에 따른 종교인소득 중 다음의 어느 하나에 해당하는 소

In [30]:
retrieved_doc['documents'][0]

['법」, 「사립학교교직원 연금법」, 「별정우체국법」 또는 「국민연금과 직역연금의 연계에 관한 법률」(이하 “공적연금 관련법”이라 한다)에 따라 받는 유족연금ㆍ퇴직유족연금ㆍ퇴역유족연금ㆍ장해유족연금ㆍ상이유족연금ㆍ순직유족연금ㆍ직무상유족연금ㆍ위험직무순직유족연금, 장애연금, 장해연금ㆍ비공무상 장해연금ㆍ비직무상 장해연금, 상이연금(傷痍年金), 연계노령유족연금 또는 연계퇴직유족연금나. 삭제<2013. 1. 1.>다. 「산업재해보상보험법」에 따라 받는 각종 연금라. 「국군포로의 송환 및 대우 등에 관한 법률」에 따른 국군포로가 받는 연금마. 삭제<2013. 1. 1.>5. 기타소득 중 다음 각 목의 어느 하나에 해당하는 소득가. 「국가유공자 등 예우 및 지원에 관한 법률」 또는 「보훈보상대상자 지원에 관한 법률」에 따라 받는 보훈급여금ㆍ학습보조비 및 「북한이탈주민의 보호 및 정착지원에 관한 법률」에 따라 받는 정착금ㆍ보로금(報勞金)과 그 밖의 금품나. 「국가보안법」에 따라 받는 상금과 보로금다. 「상훈법」에 따른 훈장과 관련하여 받는 부상(副賞)이나 그 밖에 대통령령으로 정하는 상금과 부상라. 종업원등 또는 대학의 교직원이 퇴직한 후에 사용자등 또는 산학협력단으로부터 지급받거나 대학의 학생이 소속 대학에 설치된 산학협력단으로부터 받는 직무발명보상금으로서 대통령령으로 정하는 금액. 다만, 직무발명보상금을 지급한 사용자등 또는 산학협력단과 대통령령으로 정하는 특수관계에 있는 자가 받는 직무발명보상금은 제외한다.마. 「국군포로의 송환 및 대우 등에 관한 법률」에 따라 국군포로가 받는 위로지원금과 그 밖의 금품바. 「문화유산의 보존 및 활용에 관한 법률」에 따라 국가지정문화유산으로 지정된 서화ㆍ골동품의 양도로 발생하는 소득사. 서화ㆍ골동품을 박물관 또는 미술관에 양도함으로써 발생하는 소득아. 제21조제1항제26호에 따른 종교인소득 중 다음의 어느 하나에 해당하는 소득\u3000\u3000\u3000\u30001) 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준직업분류에 따

## llm 전달

In [31]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [32]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요. {retrieved_doc['documents'][0]}"},
        {"role": "user", "content": query}
    ]
)

In [33]:
response

ChatCompletion(id='chatcmpl-BFFt06ZhFNPY4ndhD7WJO40BMxJmZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='연봉 5천만 원인 직장인의 소득세를 계산하기 위해서는 한국의 소득세 과세표준과 세율, 그리고 공제 항목을 고려해야 합니다. 2023년의 소득세율 기준으로 기본적인 세율 체계는 다음과 같습니다.\n\n1. 1,200만 원 이하는 6%\n2. 1,200만 원 초과 4,600만 원 이하는 15%\n3. 4,600만 원 초과 8,800만 원 이하는 24%\n4. 8,800만 원 초과 1억 5천만 원 이하는 35%\n5. 1억 5천만 원 초과 3억 원 이하는 38%\n6. 3억 원 초과는 40%\n\n연봉 5천만 원의 소득세는 다음과 같이 계산할 수 있습니다:\n\n1. 1,200만 원까지: 1,200만 원 * 6% = 72만 원\n2. 1,200만 원 초과 4,600만 원까지: (4,600만 원 - 1,200만 원) * 15% = 510만 원\n3. 4,600만 원 초과 5,000만 원까지: (5,000만 원 - 4,600만 원) * 24% = 96만 원\n\n따라서 소득세는 72만 원 + 510만 원 + 96만 원 = 678만 원입니다.\n\n하지만 이는 기본적인 소득세 계산이며, 실제 납부해야 할 세액은 근로소득 공제, 인적 공제 등 다양한 공제를 반영해야 합니다. 구체적인 공제 항목에 따라 최종 세액은 달라질 수 있으므로, 정확한 세액을 계산하려면 이러한 공제를 모두 고려해야 합니다. 특히, 근로소득자는 근로소득 공제와 기본공제 외에 추가적으로 특별소득공제나 세액공제를 받을 수 있으므로, 국세청 홈택스 사이트 등의 소득세 계산기를 통해 상세한 정보를 입력하여 계산하는 것이 좋습니다.', refusal=None, role='assistant', annotations=[], audi

In [34]:
response.choices[0].message.content

'연봉 5천만 원인 직장인의 소득세를 계산하기 위해서는 한국의 소득세 과세표준과 세율, 그리고 공제 항목을 고려해야 합니다. 2023년의 소득세율 기준으로 기본적인 세율 체계는 다음과 같습니다.\n\n1. 1,200만 원 이하는 6%\n2. 1,200만 원 초과 4,600만 원 이하는 15%\n3. 4,600만 원 초과 8,800만 원 이하는 24%\n4. 8,800만 원 초과 1억 5천만 원 이하는 35%\n5. 1억 5천만 원 초과 3억 원 이하는 38%\n6. 3억 원 초과는 40%\n\n연봉 5천만 원의 소득세는 다음과 같이 계산할 수 있습니다:\n\n1. 1,200만 원까지: 1,200만 원 * 6% = 72만 원\n2. 1,200만 원 초과 4,600만 원까지: (4,600만 원 - 1,200만 원) * 15% = 510만 원\n3. 4,600만 원 초과 5,000만 원까지: (5,000만 원 - 4,600만 원) * 24% = 96만 원\n\n따라서 소득세는 72만 원 + 510만 원 + 96만 원 = 678만 원입니다.\n\n하지만 이는 기본적인 소득세 계산이며, 실제 납부해야 할 세액은 근로소득 공제, 인적 공제 등 다양한 공제를 반영해야 합니다. 구체적인 공제 항목에 따라 최종 세액은 달라질 수 있으므로, 정확한 세액을 계산하려면 이러한 공제를 모두 고려해야 합니다. 특히, 근로소득자는 근로소득 공제와 기본공제 외에 추가적으로 특별소득공제나 세액공제를 받을 수 있으므로, 국세청 홈택스 사이트 등의 소득세 계산기를 통해 상세한 정보를 입력하여 계산하는 것이 좋습니다.'